In [1]:
import pandas as pd 

In [4]:
data_smiles = pd.read_excel('data/Huang_Massa_data_with_all_SMILES.xlsx', skipfooter=1)
data_smiles

,num,Molecular Name,Formula,Density (g/cm3),Delta Hf solid (kj/mol),Explosive energy (kj/cc),Shock velocity (km/s),Particle velocity (km/s),Speed of sound (km/s),Pressure (Gpa),T(K),TNT Equiv (per cc),group,SMILES,NOTE
0,1,CUGCOW,C8N7O14H1,2.024,641.16,6.419,9.569,2.519,7.050,48.777,5440,2.233,Cubane,O=N(=O)C12C3C4(N(=O)=O)C1(N(=O)=O)C1(N(=O)=O)C...,NaN
1,2,PUBMUU02,C6N12O12H6,2.044,430.83,5.623,9.972,2.346,7.626,47.807,3886,1.828,CL20,O=N(=O)N1C2C3N(N(=O)=O)C4C1N(N(=O)=O)C(C(N2N(=...,NaN
2,3,KURT08MO,C4N8O10H4F4,2.028,-151.80,5.290,8.365,2.286,6.079,38.780,4342,1.842,Linear,[H]C([H])(N(N(=O)=O)C([H])([H])C(N(F)F)(N(=O)=...,NaN
3,4,ELUDEV,C4N10O6H2,1.996,657.13,5.228,9.729,2.289,7.441,44.490,3950,1.698,Pyrazole,O=N(=O)NC1=NON=C1N=NC1=NON=C1NN(=O)=O,NaN
4,5,UGUGUY,C8N6O12H2,1.931,593.87,5.188,9.061,2.318,6.743,40.555,5184,1.920,Cubane,O=N(=O)C12C3C4(C1C1(N(=O)=O)C2(N(=O)=O)C3(N(=O...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,102,ZZZMUC,C7N3O6H5,1.654,-63.54,2.359,7.179,1.689,5.491,20.053,3244,1.000,TNT,Cc1c(cc(cc1N(=O)=O)N(=O)=O)N(=O)=O,TNT - with isolated molecule refcode
105,64,CHATC3N,C4N4O7H4,1.952,-435.58,3.829,8.650,1.981,6.669,33.443,2809,1.226,Butterfly,[H]C2([H])N(N(=O)=O)C1([H])OC(=O)OC1([H])N2N(=...,"Note: this one is cut off at the bottom, but f..."
106,68,SRIP25,CN6OH4,1.725,325.49,3.768,9.384,2.090,7.294,33.831,2684,1.261,Pyrazole,[H]ON([H])n1cnc(N([H])[H])n1,Note the discrepancy between the chemical form...
107,93,CLCPI,C7N8O10H10,1.756,-174.72,3.229,8.074,1.918,6.156,27.188,3150,1.226,Butterfly,[H]C([H])([H])[O]COC2C(N(=O)=O)C1C(N(N(=O)=O)C...,"""Me"" is short for a methyl - a carbon with 3 ..."
